In [16]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices(device_type='GPU') #本地需要这样操作
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu ,True)

In [17]:
dataset=tf.data.Dataset.from_tensor_slices([1,2,3,4,5])
dataset

<TensorSliceDataset shapes: (), types: tf.int32>

In [18]:
for e in dataset:
    print(e,e.numpy())
#e.numpy() tensor转化为numpy

tf.Tensor(1, shape=(), dtype=int32) 1
tf.Tensor(2, shape=(), dtype=int32) 2
tf.Tensor(3, shape=(), dtype=int32) 3
tf.Tensor(4, shape=(), dtype=int32) 4
tf.Tensor(5, shape=(), dtype=int32) 5


In [19]:
dataset=tf.data.Dataset.from_tensor_slices([[1,2],[3,4],[5,6]])
dataset

<TensorSliceDataset shapes: (2,), types: tf.int32>

In [20]:
for e in dataset:
    print(e,e.numpy())

tf.Tensor([1 2], shape=(2,), dtype=int32) [1 2]
tf.Tensor([3 4], shape=(2,), dtype=int32) [3 4]
tf.Tensor([5 6], shape=(2,), dtype=int32) [5 6]


In [21]:
dataset_dict=tf.data.Dataset.from_tensor_slices(
    {
        'a':[1,2,3,4],
        'b':[5,6,7,8],
        'c':[9,10,11,12]
    }
)
dataset_dict

<TensorSliceDataset shapes: {a: (), b: (), c: ()}, types: {a: tf.int32, b: tf.int32, c: tf.int32}>

In [22]:
for e in dataset_dict.take(2):
    print(e)

{'a': <tf.Tensor: shape=(), dtype=int32, numpy=1>, 'b': <tf.Tensor: shape=(), dtype=int32, numpy=5>, 'c': <tf.Tensor: shape=(), dtype=int32, numpy=9>}
{'a': <tf.Tensor: shape=(), dtype=int32, numpy=2>, 'b': <tf.Tensor: shape=(), dtype=int32, numpy=6>, 'c': <tf.Tensor: shape=(), dtype=int32, numpy=10>}


In [23]:
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.mnist.load_data()

x_train=x_train/255
x_test=x_test/255
x_train_ds=tf.data.Dataset.from_tensor_slices(x_train)
y_train_ds=tf.data.Dataset.from_tensor_slices(y_train)
x_train_ds, y_train_ds

(<TensorSliceDataset shapes: (28, 28), types: tf.float64>,
 <TensorSliceDataset shapes: (), types: tf.uint8>)

In [24]:
x_test_ds=tf.data.Dataset.from_tensor_slices(x_train)
y_test_ds=tf.data.Dataset.from_tensor_slices(y_train)


In [25]:
dataset_test=tf.data.Dataset.zip((x_test_ds,y_test_ds))
dataset_train=tf.data.Dataset.zip((x_train_ds,y_train_ds))


In [26]:
dataset_train

<ZipDataset shapes: ((28, 28), ()), types: (tf.float64, tf.uint8)>

In [27]:
dataset_train=dataset_train.shuffle(10000).repeat().batch(64)
dataset_test=dataset_test.batch(64).repeat() #不设置不会默认repeat

In [28]:
model=tf.keras.Sequential(
    [
        tf.keras.layers.Flatten(input_shape=(28,28)),
        tf.keras.layers.Dense(128,activation='relu'),
        tf.keras.layers.Dense(10,activation='softmax')
    ]
)


In [29]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=['acc']
              )


In [30]:
batch_size=64
#//保证为整数
steps_per_epoch=x_train.shape[0]//batch_size
validation_steps=x_test.shape[0]//batch_size
model.fit(
    dataset_train,
    epochs=5,
    steps_per_epoch=steps_per_epoch,
    validation_data=dataset_test,
    validation_steps=validation_steps
)




Train for 937 steps, validate for 156 steps
Epoch 1/5
937/937 [==============================] - 3s 3ms/step - loss: 0.2971 - acc: 0.9166 - val_loss: 0.1602 - val_acc: 0.9555